I will be conducting a between design Factorial Analysis of Variance (ANOVA)/two-way ANOVA with two independent variables which are temperature (positive or negative) and national unemployment (low, medium, high) to look at their effects on weekly sales. Since this is a 2X3 design, there will be a total of six conditions. This dataset contains data released by Walmart and it can be found at https://www.kaggle.com/datasets/yasserh/walmart-dataset  

In [1]:
library(readr)
walmart <- read_csv("Walmart.csv")
walmart <- as.data.frame(walmart)
dim(walmart)
head(walmart)

Rows: 6435 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): Date
dbl (7): Store, Weekly_Sales, Holiday_Flag, Temperature, Fuel_Price, CPI, Un...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 6435    8

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,05-02-2010,1643691,0,42.31,2.572,211.0964,8.106
2,1,12-02-2010,1641957,1,38.51,2.548,211.2422,8.106
3,1,19-02-2010,1611968,0,39.93,2.514,211.2891,8.106
4,1,26-02-2010,1409728,0,46.63,2.561,211.3196,8.106
5,1,05-03-2010,1554807,0,46.50,2.625,211.3501,8.106
6,1,12-03-2010,1439542,0,57.79,2.667,211.3806,8.106


In [ ]:
dim(walmart)

In [ ]:
any(is.na(walmart)) # to check if there are any missing values

[1] FALSE

In [2]:
walmart$Temperature <- (walmart$Temperature - 32) * 5/9 # convert temperature from fahrenheit to celcius
walmart$temperature_category <- walmart$Temperature

I will use a for loop to transform the temperature column which currently holds numerical data in to a factor column that holds categorical string data. This new column will be named temperature_category.

In [3]:
for(i in 1:length(walmart$temperature_category)){
  if(walmart$temperature_category[i] >= 0){
    walmart$temperature_category[i] <- "Positive_Temperature"
  } else {
    walmart$temperature_category[i] <- "Negative_Temperature"
  }
}

walmart$temperature_category <- as.factor(walmart$temperature_category)

In [ ]:
head(walmart)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,temperature_category
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1,05-02-2010,1643691,0,5.727778,2.572,211.0964,8.106,Positive_Temperature
2,1,12-02-2010,1641957,1,3.616667,2.548,211.2422,8.106,Positive_Temperature
3,1,19-02-2010,1611968,0,4.405556,2.514,211.2891,8.106,Positive_Temperature
4,1,26-02-2010,1409728,0,8.127778,2.561,211.3196,8.106,Positive_Temperature
5,1,05-03-2010,1554807,0,8.055556,2.625,211.3501,8.106,Positive_Temperature
6,1,12-03-2010,1439542,0,14.327778,2.667,211.3806,8.106,Positive_Temperature


The first categorical independent variable has been created, I will now create a second categorical independent variable (in the "cat" column) by using a for loop again. This new categorical variable/column will use the Unemployment column which contains the national unemployment rate for every given date, to create three levels of unemployment. These levels are low, medium, and high.

In [4]:

for(i in 1:length(walmart$Unemployment)){
  if(walmart$Unemployment[i] <= 5.0){
    walmart$cat[i] <- "Low"
  } else if(walmart$Unemployment[i] <= 10.0 ){
    walmart$cat[i] <- "Medium"
  } else {
    walmart$cat[i] <- "High"
  }
}


In [ ]:
head(walmart)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,temperature_category,cat
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
1,1,05-02-2010,1643691,0,5.727778,2.572,211.0964,8.106,Positive_Temperature,Medium
2,1,12-02-2010,1641957,1,3.616667,2.548,211.2422,8.106,Positive_Temperature,Medium
3,1,19-02-2010,1611968,0,4.405556,2.514,211.2891,8.106,Positive_Temperature,Medium
4,1,26-02-2010,1409728,0,8.127778,2.561,211.3196,8.106,Positive_Temperature,Medium
5,1,05-03-2010,1554807,0,8.055556,2.625,211.3501,8.106,Positive_Temperature,Medium
6,1,12-03-2010,1439542,0,14.327778,2.667,211.3806,8.106,Positive_Temperature,Medium


In [5]:
walmart$cat <- as.factor(walmart$cat)
levels(walmart$cat)

sum(as.numeric(walmart$cat == "Low"))
sum(as.numeric(walmart$cat == "Medium"))
sum(as.numeric(walmart$cat == "High"))

[1] "High"   "Low"    "Medium"

[1] 213

[1] 5593

[1] 629

I will now use the Levene's Test to check for homogeneity/equality of variances and use the the Kolmogorov-Smirnov Test to check if the data is normally distributed. Since the size of walmart$Weekly_Sales is greater than 5000, we will not be to use the Shapiro-Wilk Normality test.

In [ ]:
install.packages('car')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘Rcpp’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’




In [ ]:
library('car')
leveneTest(Weekly_Sales ~ temperature_category*cat, walmart)

,Df,F value,Pr(>F)
,<int>,<dbl>,<dbl>
group,5,36.28638,8.920948e-37
,6429,NA,NA


In [ ]:
# Since the size of walmart$Weekly_Sales is greater than 5000, we will not be to use the Shapiro-Wilk Normality test. Therefore, the Kolmogorov-Smirnov Test will be used instead
ks.test(walmart$Weekly_Sales, 'pnorm')


	Asymptotic one-sample Kolmogorov-Smirnov test

data:  walmart$Weekly_Sales
D = 1, p-value < 2.2e-16
alternative hypothesis: two-sided


Both equality/homogeneity of variance and assumption of normality are violated. Therefore, a non-parametric test should be used that can analyze the effects of two independent variables (Temperature and Unemployment) on the dependent variable. However, for demonstration purposes, I will proceed to conduct a 2X3 between-design Factorial ANOVA.

In [ ]:
summary(aov(Weekly_Sales ~ temperature_category + cat + temperature_category:cat, walmart))

                           Df    Sum Sq   Mean Sq F value Pr(>F)    
temperature_category        1 8.746e+09 8.746e+09   0.028 0.8667    
cat                         2 5.027e+13 2.514e+13  80.919 <2e-16 ***
temperature_category:cat    2 1.937e+12 9.683e+11   3.117 0.0443 *  
Residuals                6429 1.997e+15 3.106e+11                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

There is no significant effect of temperature on weekly sales but there is a significant effect of unemployment category and also a signficant interaction between temperature and unemployment category. The signficant interactions means that the effect of either IV is not the same at different levels of the other IV.

  Therefore, we will do a simple effects analysis looking at the effect of unemployment category at different levels of temperature.

In [6]:
# at the positive level of the temperature variable
summary(aov(walmart$Weekly_Sales[walmart$temperature_category == 'Positive_Temperature'] ~
walmart$cat[walmart$temperature_category == 'Positive_Temperature']))

                                                                      Df
walmart$cat[walmart$temperature_category == "Positive_Temperature"]    2
Residuals                                                           5917
                                                                       Sum Sq
walmart$cat[walmart$temperature_category == "Positive_Temperature"] 4.973e+13
Residuals                                                           1.835e+15
                                                                      Mean Sq
walmart$cat[walmart$temperature_category == "Positive_Temperature"] 2.486e+13
Residuals                                                           3.101e+11
                                                                    F value
walmart$cat[walmart$temperature_category == "Positive_Temperature"]   80.17
Residuals                                                                  
                                                                    Pr(>F)    
walmar

Unemployment has a signficant effect during positive temperature days which means that weekly sales signficantly differ among the levels of unemployment (low, medium,high) on positive temperature days.

  To find out which groups signficantly differ we will use the Scheffe post-hoc test.


In [ ]:
install.packages("DescTools")

In [ ]:
library(DescTools)

In [ ]:
ScheffeTest(aov(walmart$Weekly_Sales[walmart$temperature_category == 'Positive_Temperature'] ~
walmart$cat[walmart$temperature_category == 'Positive_Temperature']))


  Posthoc multiple comparisons of means: Scheffe Test 
    95% family-wise confidence level

$`walmart$cat[walmart$temperature_category == "Positive_Temperature"]`
                 diff    lwr.ci    upr.ci    pval    
Low-High     563371.9  449399.7  677344.0 < 2e-16 ***
Medium-High  211164.0  152763.2  269564.9 < 2e-16 ***
Medium-Low  -352207.8 -453720.3 -250695.3 2.7e-16 ***

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


All groups siginficantly differ from each other. Below I will create a new data frame to find the mean for each group in order to find the nature of the differences.

In [ ]:
o1 <- walmart$Weekly_Sales[walmart$temperature_category == 'Positive_Temperature']
o2 <- walmart$cat[walmart$temperature_category == 'Positive_Temperature']
dataf <- data.frame(o1,o2)
head(dataf)

,o1,o2
,<dbl>,<fct>
1,1643691,Medium
2,1641957,Medium
3,1611968,Medium
4,1409728,Medium
5,1554807,Medium
6,1439542,Medium


In [ ]:
mean(dataf$o1[dataf$o2 == "Low"]) # mean for low unemployment
mean(dataf$o1[dataf$o2 == "Medium"]) # mean for medium unemployment
mean(dataf$o1[dataf$o2 == "High"]) # mean for high unemployment

[1] 1409462

[1] 1057254

[1] 846089.9

The mean for high unemployment is the highest followed by the mean medium unemployment which is the second highest and the mean for low unemployment is the lowest. Since all groups signficantly differ from each other, we can conclude that sales are the highest when unemployment is highest with sales decreasing as unemployment decreases. This is only for positive temperature days.

In [9]:
# at the negative level of the temperature variable
summary(aov(walmart$Weekly_Sales[walmart$temperature_category == 'Negative_Temperature'] ~
walmart$cat[walmart$temperature_category == 'Negative_Temperature']))

                                                                     Df
walmart$cat[walmart$temperature_category == "Negative_Temperature"]   2
Residuals                                                           512
                                                                       Sum Sq
walmart$cat[walmart$temperature_category == "Negative_Temperature"] 2.481e+12
Residuals                                                           1.620e+14
                                                                      Mean Sq
walmart$cat[walmart$temperature_category == "Negative_Temperature"] 1.241e+12
Residuals                                                           3.164e+11
                                                                    F value
walmart$cat[walmart$temperature_category == "Negative_Temperature"]   3.921
Residuals                                                                  
                                                                    Pr(>F)  
walmart$cat

Unemployment has a signficant effect during negative temperature days as well which means that weekly sales signficantly differ among the levels of unemployment (low, medium,high) on negative temperature days as well.

To find out which groups signficantly differ we will use the Scheffe post-hoc test.

In [ ]:
ScheffeTest(aov(walmart$Weekly_Sales[walmart$temperature_category == 'Negative_Temperature'] ~
walmart$cat[walmart$temperature_category == 'Negative_Temperature']))


  Posthoc multiple comparisons of means: Scheffe Test 
    95% family-wise confidence level

$`walmart$cat[walmart$temperature_category == "Negative_Temperature"]`
                 diff     lwr.ci   upr.ci   pval    
Low-High    429974.53   13227.69 846721.4 0.0413 *  
Medium-High 353276.83   30161.01 676392.7 0.0280 *  
Medium-Low  -76697.69 -354883.37 201488.0 0.7954    

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


We can see that the mean sales for medium and low unemployment do not signficantly differ. But both of them are signficantly different from mean sales of high unemployment.

I will now calculate the group means below.

In [ ]:
o3 <- walmart$Weekly_Sales[walmart$temperature_category == 'Negative_Temperature']
o4 <- walmart$cat[walmart$temperature_category == 'Negative_Temperature']
dataf2 <- data.frame(o3,o4)
head(dataf2)

,o3,o4
,<dbl>,<fct>
1,2188307.4,Medium
2,1865502.5,Medium
3,2508955.2,Medium
4,496725.4,Medium
5,524104.9,Medium
6,506760.5,Medium


In [ ]:
mean(dataf2$o3[dataf2$o4 == "Low"]) # mean for low unemployment
mean(dataf2$o3[dataf2$o4 == "Medium"]) # mean for medium unemployment
mean(dataf2$o3[dataf2$o4 == "High"]) # mean for high unemployment

[1] 1136777

[1] 1060079

[1] 706802.1

We can conclude that for negative temperature days, mean sales are highest for high unemployment similar to positive temperature days. However, unlike positive temperature days, on negative temperature days mean sales do not decrease when unemployment rate goes from medium to low.

Therefore, the effect of unemployment rate on weekly sales is signficant overall with higher unemployment associated with higher mean sales. But the effect of unemployment is not the same at all levels of temperature.